In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy.stats
import matplotlib.pyplot as plt

from matplotlib import animation
from matplotlib import rcParams
rcParams['figure.dpi'] = 120
from IPython.display import HTML
from IPython.display import YouTubeVideo
from functools import partial
YouTubeVideo_formato = partial(YouTubeVideo, modestbranding=1, disablekb=0,
                               width=640, height=360, autoplay=0, rel=0, showinfo=0)

# Test de hipótesis

Se aplica un tratamiento nuevo a una muestra de la población 

- ¿Es el tratamiento efectivo?
- ¿Existe una diferencia entre los que tomaron el tratamiento y los que no?

El test de hipótesis es un procedimiento estadístico **para comprobar si el resultado de un experimento es significativo en la población**

Para esto formulamos dos escenarios cada uno con una hipótesis asociada

- Hipótesis nula ($H_0$): Por ejemplo
    - "El experimento no produjo diferencia"
    - "El experimento no tuvo efecto"
    - "Las observaciones son producto del azar"
- Hipótesis alternativa ($H_A$): Usualmente el complemento de $H_0$

:::{important}

El test de hipótesis se diseña para medir que tan fuerte es la evidencia **en contra** de la hipótesis nula

:::

## Algoritmo general de un test de hipótesis

El siguiente es el algoritmo general de un test de hipótesis paramétrico

1. Definimos $H_0$ y $H_A$
1. Definimos un estadístico $T$
1. Asumimos una distribución para $T$ dado que $H_0$ es cierto
1. Seleccionamos un nivel de significancia $\alpha$ 
1. Calculamos el $T$ para nuestros datos $T_{data}$
1. Calculamos el **p-value**
    - Si nuestro test es de una cola:
        - Superior: $p = P(T>T_{data})$
        - Inferior: $p = P(T<T_{data})$
    - Si nuestro test es dos colas: $p = P(T>T_{data}) + P(T<T_{data})$

Finalmente:

`Si`  $p < \alpha$
    
> Rechazamos la hipótesis nula con confianza (1-$\alpha$)

`De lo contrario`
    
> No hay suficiente evidencia para rechazar la hipótesis nula

El valor de $\alpha$ nos permite controlar el **[Error tipo I](https://es.wikipedia.org/wiki/Errores_de_tipo_I_y_de_tipo_II)**, es decir el error que cometemos si rechazamos $H_0$ cuando en realidad era cierta (falso positivo)

:::{note}

Típicamente se usa $\alpha=0.05$ o $\alpha=0.01$ 

:::

**Errores de interpretación comunes**

Muchas veces se asume que el p-value es la probabilidad de que $H_0$ sea cierta dado nuestras observaciones

$$
p = P(H_0 | T> T_{data})
$$

Esto es un **grave error**. Formálmente el **p-value** es la probabilidad de observar un valor de $T$ más extremo que el observado, es decir 

$$
p = P(T> T_{data} | H_0) 
$$

Otro error común es creer que no ser capaz de rechazar $H_0$ es lo mismo que aceptar $H_0$

:::{error}

No tener suficiente evidencia para rechazar no es lo mismo que aceptar

:::

## El t-test de una muestra 

Sea un conjunto de $N$ observaciones iid $X = {x_1, x_2, \ldots, x_N}$ con media muestral $\bar x = \sum_{i=1}^N x_i$ 

El t-test de una muestra es busca verificar si $\bar x$ es significativamente distinta de la **media poblacional** $\mu$, en el caso de que **no conocemos la varianza poblacional** $\sigma^2$

Las hipótesis son

- $H_0:$ $\bar x = \mu$
- $H_A:$ $\bar x \neq \mu$ (dos colas)

El estadístico de prueba es 

$$
t = \frac{\bar x - \mu}{\hat \sigma /\sqrt{N-1}}
$$

donde $\hat \sigma = \sqrt{ \frac{1}{N} \sum_{i=1}^N (x_i - \bar x)^2}$ es la desviación estándar muestral (sesgada)

Si asumimos que $\bar x$ se distribuye $\mathcal{N}(\mu, \frac{\sigma^2}{N})$ entonces
$t$ se distribuye [t-student](https://en.wikipedia.org/wiki/Student%27s_t-distribution) con $N-1$ grados de libertad

- Para muestras iid y $N$ grande el supuesto se cumple por teorema central del límite
- Si $N$ es pequeño debemos verificar la normalidad de los datos


**Ejemplo: Aplicación de t-test para probar que la regresión es significativa**

En un modelo de regresión lineal donde tenemos $N$ ejemplos

$$
y_i = x_i \theta_1 + \theta_0, ~ i=1, 2, \ldots, N
$$

Podemos probar que la correlación entre $x$ es $y$ es significativa realizando un test sobre el parámetro $\theta_1$

Por ejemplo podemos plantear las siguientes hipótesis

- $H_0:$ La pendiente es nula $\theta_1= 0$ 
- $H_A:$ La pendiente no es nula: $\theta_1\neq 0$ (dos colas)

Y asumiremos que $\theta_1$ es normal pero que desconocemos su varianza. Bajo este supuesto se puede formular el siguiente estadístico de prueba 

$$
t = \frac{(\theta_1-\theta^*) }{\text{SE}_{\theta_1}/\sqrt{N-2}} = \frac{ r\sqrt{N-2}}{\sqrt{1-r^2}},
$$

donde $r$ es el coeficiente de correlación de Pearson (detalles más adelante) y la última expresión se obtiene reemplazando  $\theta^*=0$ y 

$$
\text{SE}_{\theta_1} = \sqrt{ \frac{\frac{1}{N} \sum_i (y_i - \hat y_i)^2}{\text{Var}(x)}}.
$$ 

Este estadístico tiene distribución t-student con dos grados de libertad (modelo de dos parámetros) 


**Ejercicio formativo**

Sea el dataset de consumo de helados que utilizamos en lecciones anteriores

In [ ]:
df = pd.read_csv('../data/helados.csv', header=0, index_col=0)
df.columns = ['consumo', 'ingreso', 'precio', 'temperatura']
display(df.head())

In [ ]:

En lecciones anteriores estudiamos el modelo de regresión lineal el cual nos permite estudiar si existe correlación entre variables continuas. También vimos como ajustar los parámetros del modelo usando el método de mínimos cuadrados. En este ejercicio formativo veremos como verificar si el modelo de regresión ajustado es correcto

Luego de revisar este ejercicio usted habrá aprendido

- La interpretación probabilística de la regresión lineal y la relación entre mínimos cuadrados ordinarios y la estimación por máxima verosimilitud
- El estadístico $r$ para medir la fuerza de la correlación entre dos variables
- Un test de hipótesis para verificar que la correlación encontrada es estadística significativa

Usaremos el dataset de consumo de helados


A continuación se muestra un gráfico de dispersión del consumo en función de las demás variables. ¿Cree usted que existe correlación en este caso?

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(8, 3), tight_layout=True, sharey=True)
for i, col in enumerate(df.columns[1:]):
    ax[i].scatter(df[col], df["consumo"], s=10)
    ax[i].set_xlabel(col)
ax[0].set_ylabel(df.columns[0]);

### Interpretación probabilística y MLE de la regresión lineal

Sea $y$ el consumo y $x$ la temperatura.

Asumiremos errores gaussianos iid

$$
y_i = \hat y_i + \epsilon_i, \epsilon_i \sim \mathcal{N}(0, \sigma^2),
$$

y un modelo lineal de **dos parámetros** (linea recta)

$$
\hat y_i = \theta_0 + \theta_1 x_i
$$

Bajo estos supuestos el estimador de máxima verosimilitud es 

$$
\begin{align}
\hat \theta &= \text{arg}\max_\theta \log \mathcal{L}(\theta) \nonumber \\
&=\text{arg}\max_\theta  - \frac{1}{2\sigma^2} \sum_{i=1}^N (y_i - \theta_0 - \theta_1 x_i)^2 \nonumber
\end{align}
$$

Es decir que el estimador de máxima verosimilitud es equivalente al de mínimos cuadrados ordanrios $\hat \theta= (X^T X)^{-1} X^T y$ que vimos anteriormente

**Importante:** Cuando utilizamos la solución de mínimos cuadrados estamos asumiendo implicitamente que las observaciones son iid y que la verosimilitud es Gaussiana


Derivando con respecto a los parámetros e igualado a cero tenemos que

$$
\begin{align}
\sum_i y_i  - N\theta_0 - \theta_1  \sum_i x_i &= 0 \nonumber \\
\sum_i y_i x_i - \theta_0 \sum_i x_i - \theta_1 \sum_i x_i^2 &= 0 \nonumber
\end{align}
$$

Finalmente podemos despejar

$$
\begin{align}
\theta_0 &= \bar y - \theta_1 \bar x \nonumber \\
\theta_1 &= \frac{\sum_i x_i y_i - N \bar x \bar y}{\sum_i x_i^2 - M \bar x^2}  \nonumber \\
&= \frac{ \sum_i (y_i - \bar y)(x_i - \bar x)}{\sum_i (x_i - \bar x)^2}  = \frac{\text{COV}(x, y)}{\text{Var}(x)}
\end{align}
$$

de donde reconocemos las expresiones para la covarianza entre $x$ e $y$ y la varianza de $x$


### Coeficiente de correlación de Pearson

La fuerza de la correlación se suele medir usando

$$
r^2 = 1 - \frac{\sum_i ( y_i - \hat y_i)^2}{\sum_i ( y_i - \bar y)^2} = 1 - \frac{\frac{1}{M} \sum_i (y_i - \hat y_i)^2}{\text{Var}(y)} = \frac{\text{COV}^2(x, y)}{\text{Var}(x) \text{Var}(y)}
$$

donde $r = \frac{\text{COV}(x, y)}{\sqrt{\text{Var}(x) \text{Var}(y)}} \in [-1, 1]$ se conoce como [coeficiente de correlación de Pearson](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient)

donde

- si $r=1$ existe una correlación lineal perfecta
- si $r=-1$ existe una anticorrelación lineal perfecta
- si $r=0$ no hay correlación lineal entre las variables

En general un $r>0.5$ se considera una correlación importante

**Calculando $r$ con y los parámetros de la regresión lineal**

Podemos usar el atributo de dataframe

```python
df.corr()
```

Que retorna la matriz de correlaciones lineales

In [ ]:
df.corr()

Si queremos también el valor de los parámetros podemos usar la función de scipy 

```python
scipy.stats.linregress(x, # Variable independiente unidimensional
                       y # Variable dependiente unidimensional
                      )
```

Esta función retorna una tupla con

- Valor de la pendiente: $\theta_1$
- Valor de la intercepta: $\theta_0$
- Coeficiente de correlación $r$
- p-value
- Error estándar del ajuste

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(8, 3), tight_layout=True, sharey=True)
ax[0].set_ylabel(df.columns[0]);


for i, col in enumerate(df.columns[1:]):
    res = scipy.stats.linregress(df[col], df["consumo"])
    x_plot = np.linspace(np.amin(df[col]), np. amax(df[col]), num=100)
    ax[i].scatter(df[col], df["consumo"], label='datos', s=10)    
    ax[i].plot(x_plot, res.slope*x_plot + res.intercept, lw=2, c='r', label='modelo');
    ax[i].set_xlabel(col)
    ax[i].set_title(f"$r$: {res.rvalue:0.5f}")
    ax[i].legend()

Es decir que visualmente parece existir

- una correlación positiva alta entre consumo y temperatura
- una correlación negativa moderada entre consumo y precio
- una correlación cercana a cero entre consumo e ingreso

### Test de hipótesis y conclusiones

La función `linregress` implementa el t-test sobre $\theta_1$ que vimos anteriormente. Usemos estos resultados para verificar si las correlaciones son estadísticamente significativas

In [ ]:
alpha = 0.05

for i, col in enumerate(df.columns[1:]):
    res = scipy.stats.linregress(df[col], df["consumo"])
    print(f"{col}: \t p-value:{res.pvalue:0.4f} \t ¿Menor que {alpha}?: {res.pvalue < alpha}")    

Como complemento visualizemos 

- las distribuciones bajo la hipótesis nula: linea azul
- los límites dados por $\alpha$: linea punteada negra
- El valor del observado para cada una de las variables: linea roja 

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(8, 2), tight_layout=True, sharey=True)
ax[0].set_ylabel(df.columns[0]);

N = df.shape[0]
t = np.linspace(-7, 7, num=1000)
dist = scipy.stats.t(loc=0, scale=1, df=N-2) # dos grados de libertad


for i, col in enumerate(df.columns[1:]):
    res = scipy.stats.linregress(df[col], df["consumo"])
    t_data = res.rvalue*np.sqrt(N-2)/np.sqrt(1.-res.rvalue**2)
    ax[i].plot(t, dist.pdf(t))
    ax[i].plot([dist.ppf(alpha/2)]*2, [0, np.amax(dist.pdf(t))], 'k--')
    ax[i].plot([dist.ppf(1-alpha/2)]*2, [0, np.amax(dist.pdf(t))], 'k--')
    ax[i].plot([t_data]*2, [0, np.amax(dist.pdf(t))], 'r-')
    ax[i].set_xlabel(col)    

**Conclusión**

Basado en los p-values y considerando $\alpha=0.05$

¿Qué podemos decir de las correlaciones con el consumo de helados?

> Rechazamos la hipótesis nula de que no existe correlación entre temperatura y consumo con un 95% de confianza

Para las variables ingreso y precio no existe suficiente evidencia para rechazar $H_0$

### Reflexión final

En el ejercicio anterior usamos t-test para una regresión lineal entre dos variables ¿Qué prueba puedo usar si quiero hacer regresión lineal multivariada? 

> Se puede usar [ANOVA](https://pythonfordatascience.org/anova-python/)

¿Qué pasa si...

- mis datos tienen una relación que no es lineal? 
- $\theta_1$ no es Gaussiano/normal? 
- si el ruido no es Gaussiano? 
- si el ruido es Gaussiano pero su varianza cambia en el tiempo? 

> En estos casos no se cumplen los supuestos del modelo o del test, por ende el resultado no es confiable

Si mis supuestos no se cumplen con ninguna prueba parámetrica, la opión es utilizar pruebas no-paramétricas